# 1. Delayed EntNet Sentihood (Constructor)<br>

Parametros:

* (atributo) batch_size: cantidad de ejemplos de entrenamiento utilizados en una iteracion
* (atributo) vocab_size: cantidad de palabras en el vocabulario del archivo de embeddings
* (atributo) target_len: tamaño de los objetivos (entidades)
* (atributo) aspect_len: tamaño de los aspectos
* (atributo) sentence_len: tamaño de la oracion mas larga del dataset
* (atributo) answer_size: tamaño del vector de salida (3 porque las categorias son: positivo, negativo, ninguno)
* (atributo) embedding_size: tamaño de los vectores de embedding
* (atributo) embedding_mat: conjunto de vectores de embeddings
* (atributo) update_embeddings: cambiar o no los embeddings (bool)
* softmax_mask: aplicar una mascara o no a las operaciones con softmax
* (atributo) max_grad_norm: para evitar que la gradiente crezca demasido y se produzcan NaN
* (atributo) n_keys: numero de cadenas por cada objetivo (entidad) con embedding (key)
* (atributo) tied_keys: los embeddings (keys) de los objetivos (entidades)
* (atributo) l2_final_layer: lambda de L2 Norm (minimos cuadrados)
* (atributo) initializer: forma de inicializar los pesos de la red neuronal
* (atributo) optimizer: metodo para actualizar los pesos en cada iteracion (en vez de usar gradiente decendiente)
* (atributo) session: sesion de tensorflow
* (atributo) name: nombre del modelo

In [0]:
class Delayed_EntNet_Sentihood(object):
    def __init__(self, 
        batch_size, vocab_size, target_len, aspect_len, sentence_len, 
        answer_size, embedding_size,
        weight_tying="adj",
        hops=3,
        embedding_mat=None,
        update_embeddings=False,
        softmax_mask=True,
        max_grad_norm=5.0,
        n_keys=6,
        tied_keys=[],
        l2_final_layer=0.0,
        initializer=tf.contrib.layers.xavier_initializer(),
        optimizer=tf.train.AdamOptimizer(learning_rate=1e-2),
        global_step=None,
        session=None,
        name='Delayed_EntNet_Sentihood'):

        print name

        self._batch_size = batch_size
        self._vocab_size = vocab_size
        self._target_len = target_len
        self._aspect_len = aspect_len
        self._sentence_len = sentence_len
        self._embedding_size = embedding_size
        self._answer_size = answer_size
        self._max_grad_norm = max_grad_norm
        self._init = initializer
        self._opt = optimizer
        self._global_step = global_step
        self._name = name
        self._embedding_mat = embedding_mat
        self._update_embeddings = update_embeddings

        assert len(tied_keys) <= n_keys
        self._n_keys = n_keys
        self._tied_keys = tied_keys
        self._l2_final_layer = l2_final_layer

Metodos:

* build_inputs: crea los placeholders que el modelo va a usar (Seccion 2)
* build_vars: crea las variables que el modelo va a usar (Seccion 3)
* inference_adj: (Seccion 5)

Atributos:

* self._sentences: placeholder del tamaño mas grande de una oracion en el dataset
* self._targets: placeholder del tamaño mas grande de un objetivo (entidad)
* self._aspects: placeholder del tamaño mas grande de un aspecto
* self._entnet_input_keep_prob: probabilidad de mantener un nodo en la entrada para evitar overfit
* self._entnet_output_keep_prob: probabilidad de mantener un nodo en la salida para evitar overfit
* self._entnet_state_keep_prob: probabilidad de mantener un nodo en los estados para evitar overfit
* self._final_layer_keep_prob: probabilidad de mantener un nodo en la capa final para evitar overfit

In [0]:
        self._build_inputs()
        self._build_vars()

        logits = self._inference_adj(
            self._sentences, 
            self._targets,
            self._aspects,
            self._entnet_input_keep_prob,
            self._entnet_output_keep_prob,
            self._entnet_state_keep_prob,
            self._final_layer_keep_prob,
        )

* cross entropy:
* logits: probabilidades no normalizadas

In [0]:
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, labels=tf.cast(self._answers_one_hot, tf.float32), 
            name="cross_entropy"
        )
        cross_entropy_mean = tf.reduce_mean(
            cross_entropy, name="cross_entropy_mean"
        )

regularizacion

In [0]:
        # l2 regularization
        trainable_variables = tf.trainable_variables()
        l2_loss_final_layer = 0.0
        assert self._l2_final_layer >= 0

        if self._l2_final_layer > 0:
            final_layer_weights = [ tf.nn.l2_loss(v) for v in trainable_variables
                                    if 'R:0' in v.name]
            assert len(final_layer_weights) == 1
            l2_loss_final_layer = self._l2_final_layer * tf.add_n(final_layer_weights)

perdida

In [0]:
        # loss op
        loss_op = cross_entropy_mean + l2_loss_final_layer

gradiente

In [0]:
        # gradient pipeline
        grads_and_vars = self._opt.compute_gradients(loss_op)

        grads_and_vars = [(tf.clip_by_norm(g, self._max_grad_norm), v) for g,v in grads_and_vars]
        nil_grads_and_vars = []
        
        for g, v in grads_and_vars:
            if v.name in self._nil_vars:
                nil_grads_and_vars.append((zero_nil_slot(g), v))
            else:
                nil_grads_and_vars.append((g, v))
        
        train_op = self._opt.apply_gradients(nil_grads_and_vars, global_step=self._global_step, name="train_op")

prediccion

In [0]:
        # predict ops
        predict_op = tf.argmax(logits, 1, name="predict_op")
        predict_proba_op = tf.nn.softmax(logits, name="predict_proba_op")

Atributos:

* self.loss_op = loss_op
* self.predict_op = predict_op
* self.predict_proba_op = predict_proba_op
* self.train_op = train_op

In [0]:
        # assign ops
        self.loss_op = loss_op
        self.predict_op = predict_op
        self.predict_proba_op = predict_proba_op
        self.train_op = train_op

        init_op = tf.global_variables_initializer()
        self._sess = session
        self._sess.run(init_op, feed_dict={self._input_embedding: self._embedding_mat})

# 2. Build inputs

* (atributo) sentences: placeholder de tamaño variable x tamaño mas grande de una oracion en el dataset para guardar los indices de las palabras que lo conforman <br>
Ejemplo sentihood: Max sentence len: 50

In [0]:
    def _build_inputs(self):
        self._sentences = tf.placeholder(
            tf.int32, [None, self._sentence_len], # None(tamaño variable) x tamaño maximo de una oracion
            name="sentences"
        )

* (atributo) targets: placeholder de tamaño variable x tamaño de un objetivo (entidad) para guardar los indices<br>
Ejemplo sentihood: Max target size: 1 (LOCATION1)

In [0]:
        self._targets = tf.placeholder(
            tf.int32, [None, self._target_len], # None(tamaño variable) x tamaño de un objetivo (entidad)
            name="targets"
        )

* (atributo) aspects: placeholder de tamaño variable x tamaño de un aspecto para guardar los indices<br>
Ejemplo sentihood: Max aspect size: 2 (transit-location)

In [0]:
        self._aspects = tf.placeholder(
            tf.int32, [None, self._aspect_len], # None(tamaño variable) x tamaño de un aspecto
            name="aspects"
        )

* (atributo) answers: placeholder para los indices de las salidas (positivo, negativo, ninguno)

In [0]:
        self._answers = tf.placeholder(
            tf.int32, [None], # None(tamaño variable)
            name="answers"
        )

* (atributo) answers_one_hot: representacion one hot de las salidas (positivo, negativo, ninguno)

In [0]:
        self._answers_one_hot = tf.one_hot(
            indices=self._answers, # largo variable
            depth=self._answer_size, # ancho 3 (positivo negativo, ninguno)
        )

* (atributo) input_embedding: placeholder para el conjunto de vectores de embeddings

```
_embedding_mat.shape = (7, 3)
 array([[0., 1., 2.],
       [1., 2., 3.],
       [2., 3., 4.],
       [3., 4., 5.],
       [4., 5., 6.],
       [5., 6., 7.],
       [6., 7., 8.]], dtype=float32)
 ```

In [0]:
        self._input_embedding = tf.placeholder(
            tf.float32, shape=self._embedding_mat.shape,
            name="input_embedding"
        )

* (atributo) entnet_input_keep_prob: probabilidad de mantener un nodo al ajustar los pesos de una caracteristica para evitar overfit

In [0]:
        self._entnet_input_keep_prob = tf.placeholder(
            tf.float32, shape=[], # shape=[] es un escalar
            name="entnet_input_keep_prob"
        )

* (atributo) entnet_output_keep_prob: probabilidad de mantener un nodo en la salida de la unidad RNN para evitar overfit

In [0]:
        self._entnet_output_keep_prob = tf.placeholder(
            tf.float32, shape=[], # shape=[] es un escalar
            name="entnet_output_keep_prob"
        )

* (atributo) entnet_state_keep_prob: probabilidad de mantener un nodo en el estado oculto que "alimenta" a la siguiente capa para evitar overfit

In [0]:
        self._entnet_state_keep_prob = tf.placeholder(
            tf.float32, shape=[], # shape=[] es un escalar
            name="entnet_state_keep_prob"
        )

* (atributo) final_layer_keep_prob: probabilidad de mantener un nodo en la capa final para evitar overfit

In [0]:
        self._final_layer_keep_prob = tf.placeholder(
            tf.float32, shape=[], # shape=[] es un escalar
            name="final_layer_keep_prob"
        )

# 3. Build vars<br>

Atributos:

* self._embedding: variable del conjunto de vectores de embeddings
* self._free_keys_embedding: variable con keys extra

In [0]:
    def _build_vars(self):
        with tf.variable_scope(self._name):
            self._embedding = tf.get_variable(
                name="embedding", # Nueva variable
                dtype=tf.float32,
                initializer=self._input_embedding, # Conjunto de vectores de embeddings
                trainable=self._update_embeddings, # Cambiar o no los embeddings (bool)
            )

            self._free_keys_embedding = tf.get_variable(
                name="free_keys_embedding", # Nueva variable
                dtype=tf.float32,
                shape=[self._n_keys - len(self._tied_keys), self._embedding_size], # keys extra
                initializer=self._init, # Inicializador de pesos
                trainable=True,
            )

        self._nil_vars = set([self._embedding.name])

# 4. Mask embedding<br>

Parametros:
 
 * embedding: conjunto de vectores de embeddings (vocab_size=7 x embedding_size=3)

```
 array([[0., 1., 2.],
       [1., 2., 3.],
       [2., 3., 4.],
       [3., 4., 5.],
       [4., 5., 6.],
       [5., 6., 7.],
       [6., 7., 8.]], dtype=float32)
 ```
 
Variables:
 
* vocab_size: la cantidad de elementos que existen en el vocabulario de embeddings
* embedding_size: el tamaño de los vectores de embedding

Constantes:
* embedding_mask: mascara de un cero y unos (vocab_size=7 x 1)

```
array([[0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

```

Retorna:

```
array([[0., 0., 0.],
       [1., 2., 3.],
       [2., 3., 4.],
       [3., 4., 5.],
       [4., 5., 6.],
       [5., 6., 7.],
       [6., 7., 8.]], dtype=float32)
```


In [0]:
    def _mask_embedding(self, embedding):
        vocab_size, embedding_size = self._embedding_mat.shape
        embedding_mask = tf.constant(
            value=[0 if i == 0 else 1 for i in range(vocab_size)], # El primer elemento es cero, los demas son unos
            shape=[vocab_size, 1],
            dtype=tf.float32,
            name="embedding_mask",
        )
        return embedding * embedding_mask

# 5. Inference adj<br>

Metodos:

* mask_embedding: Aplica una mascara de unos y ceros al conjunto de embeddings (Seccion 4)

Parametros:

* sentences: placeholder de tamaño variable x tamaño mas grande de una oracion en el dataset para guardar los indices
* targets: placeholder de tamaño variable x tamaño de un objetivo (entidad) para guardar los indices
* aspects: placeholder de tamaño variable x tamaño de un aspecto para guardar los indices
* entnet_input_keep_prob: probabilidad de mantener un nodo en la entrada para evitar overfit
* entnet_output_keep_prob: probabilidad de mantener un nodo en la salida para evitar overfit
* entnet_state_keep_prob: probabilidad de mantener un nodo en los estados para evitar overfit
* final_layer_keep_prob: probabilidad de mantener un nodo en la capa final para evitar overfit

In [0]:
    def _inference_adj(self, sentences, targets, aspects, 
                       entnet_input_keep_prob, entnet_output_keep_prob, 
                       entnet_state_keep_prob, final_layer_keep_prob):
    
        with tf.variable_scope(self._name):
            masked_embedding = self._mask_embedding(self._embedding)

            batch_size = tf.shape(sentences)[0] # tf.shape retorna un vector con la 'forma', en la posicion [0] esta 

* tf.nn.embedding_lookup: Se busca el embedding de los objetivos (entidades) en el conjunto de embeddings
* reduce_mean: reduce los vectores de targets_emb a su promedio en caso de objetivos (entidades) compuestos

In [0]:
            targets_emb = tf.nn.embedding_lookup(masked_embedding, targets) # targets contiene los indices
            # [None, entity_size, emb_size]
            
            targets_emb = tf.reduce_mean(
                input_tensor=targets_emb,
                axis=1,
                keep_dims=True,
            )
            # [None, 1, emb_size]

* tf.nn.embedding_lookup: Se busca el embedding de los aspectos en el conjunto de embeddings
* reduce_mean: reduce los vectores de aspects_emb a su promedio en caso de aspectos compuestos (transit-location)

In [0]:
            aspects_emb = tf.nn.embedding_lookup(masked_embedding, aspects) # aspects contiene los indices
            # [None, aspect_size, emb_size]
            
            aspects_emb = tf.reduce_mean(
                input_tensor=aspects_emb,
                axis=1,
                keep_dims=True,
            )
            # [None, 1, emb_size]

* tf.nn.embedding_lookup: Se busca el embedding de las oraciones en el conjunto de embeddings

In [0]:
            sentences_emb = tf.nn.embedding_lookup(masked_embedding, sentences) # sentences contiene los indices
            # [None, memory_size, emb_size]
    
            sentences_len = self._sentence_length(sentences_emb)
            # [None]

* tf.nn.embedding_lookup: Se busca el embedding de los key (objetivo) de cada cadena en el conjunto de embeddings
* reduce_mean: reduce los vectores de tied_keys_emb a su promedio en caso de objetivos compuestos

In [0]:
            tied_keys_emb = tf.nn.embedding_lookup(masked_embedding, self._tied_keys)
            # [len(self._tied_keys), max_key_len, emb_size]
            
            tied_keys_emb = tf.reduce_mean(
                input_tensor=tied_keys_emb,
                axis=1,
            )
            # [len(self._tied_keys), emb_size]

* free_keys_emb: variable con keys (objetivos) extra

In [0]:
            free_keys_emb = self._free_keys_embedding
            # [n_keys - len(self._tied_keys), emb_size]

* tf.concat: uniendo las variables de los key (objetivo) 

```
t1 = [[1, 2, 3], [4, 5, 6]]
t2 = [[7, 8, 9], [10, 11, 12]]
tf.concat([t1, t2], axis=0)  # [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
```



In [0]:
            keys_emb = tf.concat(
                values=[tied_keys_emb, free_keys_emb],
                axis=0,
                name="keys_emb",
            )
            # [n_keys, emb_size]

* tf.tile: replicando los key (objetivos) para cada elemento del batch

```
batch_size = 3

keys_emb =
[[ 1.,  2.],
 [ 2.,  1.]]

tf.tile =
[[[ 1.,  2.],
  [ 2.,  1.]],

 [[ 1.,  2.],
  [ 2.,  1.]],

 [[ 1.,  2.],
  [ 2.,  1.]]]
```



In [0]:
            batched_keys_emb = tf.tile(
                input=tf.expand_dims(input=keys_emb, axis=0),
                multiples=[batch_size, 1, 1]
            )
            # [None, n_keys, emb_size]

* tf.split: se coloca cada key en vectores independientes

```
[array([[1., 2.]], dtype=float32),
 array([[2., 1.]], dtype=float32)]
```

* tf.squeeze: se redimensiona el vector de keys para eliminar dimensiones extra

```
keys=[[1., 2.],
      [2., 1.]]
```



In [0]:
            keys = tf.split(keys_emb, self._n_keys, axis=0)
            # list of [1, emb_size]
    
            keys = [tf.squeeze(key, axis=0) for key in keys]
            # list of [emb_size]

* partial: crea una nueva funcion a partir de otra con parametros pre establecidos

```
def func(a,b,c,d):
    return a + b + c + d
func2 = partial(func, a='1', b='2', c='3')
print(func2('4')) # 1234
```

* activation: se usa la funcion de activacion PReLU (Seccion 18)

In [0]:
            alpha = tf.get_variable(
                name='alpha', # Nueva variable
                shape=self._embedding_size, # Tamaño de los vectores de embedding
                initializer=tf.constant_initializer(1.0)
            ) 
    
            activation = partial(prelu, alpha=alpha)

Se crea la celda de Memoria Dinamica para recorrer la oracion de izquierda a derecha

* DynamicMemoryCell: crea una celda de Memoria Dinamica (Seccion 10)
* num_blocks: cantidad de keys
* num_units_per_block: tamaño de los vectores de embeddings
* keys: embedings de los keys
* initializer: inicializador de pesos
* recurrent_initializer: inicializador de pesos
* activation: funcion de activacion (PReLU)

* zero_state: crea la forma (shape) del estado
* sentences_emb_shape: dimensiones del vector con las entradas (oraciones)
* cell_fw: celda de Memoria Dinamica
* input_keep_prob: probabilidad de dropout al ajustar los pesos de una caracteristica
* output_keep_prob: probabilidad de dropout para cada salida de la unidad RNN
* state_keep_prob: probabilidad de dropout para el estado oculto que "alimenta" a la siguiente capa
* input_size: tamaño del embedding de las palabras en la oracion

In [0]:
            cell_fw = DynamicMemoryCell(
                num_blocks=self._n_keys,
                num_units_per_block=self._embedding_size,
                keys=keys,
                initializer=self._init,
                recurrent_initializer=self._init,
                activation=activation,
            )
  
            initial_state_fw = cell_fw.zero_state(batch_size, tf.float32)
            sentences_emb_shape = sentences_emb.get_shape()
      
            cell_fw = tf.contrib.rnn.DropoutWrapper(
                cell=cell_fw,
                input_keep_prob=entnet_input_keep_prob,
                output_keep_prob=entnet_output_keep_prob,
                state_keep_prob=entnet_state_keep_prob,
                variational_recurrent=True,
                input_size=(sentences_emb_shape[2]),
                dtype=tf.float32,
            )

Se crea la celda de Memoria Dinamica para recorrer la oracion de derecha a izquierda

* DynamicMemoryCell: crea una celda de Memoria Dinamica (Seccion 10)
* num_blocks: cantidad de keys
* num_units_per_block: tamaño de los vectores de embeddings
* keys: embedings de los keys
* initializer: inicializador de pesos
* recurrent_initializer: inicializador de pesos
* activation: funcion de activacion (PReLU)

* zero_state: crea la forma (shape) del estado
* sentences_emb_shape: dimensiones del vector con las entradas (oraciones)
* cell_bw: celda de Memoria Dinamica
* input_keep_prob: probabilidad de dropout al ajustar los pesos de una caracteristica
* output_keep_prob: probabilidad de dropout para cada salida de la unidad RNN
* state_keep_prob: probabilidad de dropout para el estado oculto que "alimenta" a la siguiente capa
* input_size: tamaño del embedding de las palabras en la oracion

In [0]:
            cell_bw = DynamicMemoryCell(
                num_blocks=self._n_keys,
                num_units_per_block=self._embedding_size,
                keys=keys,
                initializer=self._init,
                recurrent_initializer=self._init,
                activation=activation,
            )
  
            initial_state_bw = cell_bw.zero_state(batch_size, tf.float32)
    
            cell_bw = tf.contrib.rnn.DropoutWrapper(
                cell=cell_bw,
                input_keep_prob=entnet_input_keep_prob,
                output_keep_prob=entnet_output_keep_prob,
                state_keep_prob=entnet_state_keep_prob,
                variational_recurrent=True,
                input_size=(sentences_emb_shape[2]),
                dtype=tf.float32,
            )

Se unen las dos celdas de Memoria Dinamica (izquierda a derecha y derecha a izquierda) y se obtienen los ultimos estados de estas

In [0]:
            (_, _), (last_state_fw, last_state_bw) = tf.nn.bidirectional_dynamic_rnn(
                cell_fw=cell_fw,
                cell_bw=cell_bw,
                inputs=sentences_emb,
                sequence_length=sentences_len,
                initial_state_fw=initial_state_fw,
                initial_state_bw=initial_state_bw,
            )

            last_state_fw, _ = tf.split(
                value=last_state_fw,
                num_or_size_splits=[
                    self._n_keys * self._embedding_size, 
                    self._n_keys * self._embedding_size,
                ],
                axis=1
            )
    
            last_state_bw, _ = tf.split(
                value=last_state_bw,
                num_or_size_splits=[
                    self._n_keys * self._embedding_size, 
                    self._n_keys * self._embedding_size,
                ],
                axis=1
            )
            # last_state_f/bw: [None, emb_size * n_keys]
            
            last_state_fw = tf.stack(
                tf.split(last_state_fw, self._n_keys, axis=1), axis=1)
            # [None, n_keys, emb_size]
            
            last_state_bw = tf.stack(
                tf.split(last_state_bw, self._n_keys, axis=1), axis=1)
            # [None, n_keys, emb_size]
            
            last_state = last_state_fw + last_state_bw
            # [None, n_keys, emb_size]

<img src="https://i.ibb.co/y6fTK0q/liu2018-output.png" alt="liu2018-output" border="0">

In [0]:
            asp_att = tf.concat(values=[targets_emb, aspects_emb], axis=2)
            # [None, 1, emb_size * 2]
      
            W_asp_att = tf.get_variable(
                name='W_asp_att',
                shape=[self._embedding_size, self._embedding_size * 2],
                dtype=tf.float32,
                initializer=self._init,
            )
        
            temp = tf.tensordot(
                batched_keys_emb, W_asp_att, [[2], [0]]
            )
            # [None, n_keys, emb_size * 2]
            
            attention = tf.reduce_sum(temp * asp_att, axis=2)
            # [None, n_keys]
            
            attention_max = tf.reduce_max(attention, axis=-1, keep_dims=True)
            # [None, 1]
            
            attention = tf.nn.softmax(attention - attention_max)
            # [None, n_keys]
            
            attention = tf.expand_dims(attention, axis=2)
            # [None, n_keys, 1]

            u = tf.reduce_sum(last_state * attention, axis=1)
            # [None, emb_size]

<img src="https://i.ibb.co/f2njYfY/liu2018-y.png" alt="liu2018-y" border="0">

In [0]:
            R = tf.get_variable('R', [self._embedding_size, self._answer_size])
            H = tf.get_variable('H', [self._embedding_size, self._embedding_size])

            a = tf.squeeze(aspects_emb, axis=1)
            # [None, emb_size]
            
            hidden = activation(a + tf.matmul(u, H))
            # [None, emb_size]
            
            hidden = tf.nn.dropout(x=hidden, keep_prob=final_layer_keep_prob)
            # [None, emb_size]
            
            y = tf.matmul(hidden, R)
            # [None, 1] -> El elemento unico (1) es un vector de (1x3) para las categorias (positivo, negativo, ninguno)

            return y

# 6. Get mini batch start end<br>

Parametros:

* n_train: 
* batch_size: 

In [0]:
    def _get_mini_batch_start_end(self, n_train, batch_size=None):
        '''
        Args:
            n_train: int, number of training instances
            batch_size: int (or None if full batch)
        
        Returns:
            batches: list of tuples of (start, end) of each mini batch
        '''
        mini_batch_size = n_train if batch_size is None else batch_size
        batches = zip(
            range(0, n_train, mini_batch_size),
            list(range(mini_batch_size, n_train, mini_batch_size)) + [n_train]
        )
        return batches

# 7. Fit

Parametros:

* sentences: placeholder del tamaño mas grande de una oracion en el dataset
* targets: placeholder del tamaño mas grande de un objetivo (entidad)
* aspects: placeholder del tamaño mas grande de un aspecto
* entnet_input_keep_prob: probabilidad de mantener un nodo en la entrada para evitar overfit
* entnet_output_keep_prob: probabilidad de mantener un nodo en la salida para evitar overfit
* entnet_state_keep_prob: probabilidad de mantener un nodo en los estados para evitar overfit
* final_layer_keep_prob: probabilidad de mantener un nodo en la capa final para evitar overfit
* batch_size: cantidad de ejemplos de entrenamiento utilizados en una iteracion



In [0]:
    def fit(self, sentences, targets, aspects, answers, entnet_input_keep_prob, 
            entnet_output_keep_prob, entnet_state_keep_prob, 
            final_layer_keep_prob, batch_size=None):
    
        assert len(sentences) == len(targets)
        assert len(sentences) == len(aspects)
        assert len(sentences) == len(answers)
        
        batches = self._get_mini_batch_start_end(len(sentences), batch_size)
        total_loss = 0.
        
        for start, end in batches:
            feed_dict = {
                self._sentences: sentences[start:end], 
                self._targets: targets[start:end],
                self._aspects: aspects[start:end],
                self._answers: answers[start:end], 
                self._entnet_input_keep_prob: entnet_input_keep_prob,
                self._entnet_output_keep_prob: entnet_output_keep_prob,
                self._entnet_state_keep_prob: entnet_state_keep_prob,
                self._final_layer_keep_prob: final_layer_keep_prob,
            }
            loss, _ = self._sess.run(
                [self.loss_op, self.train_op], 
                feed_dict=feed_dict
            )
            total_loss = loss * len(sentences[start:end])
        return total_loss

# 8. Predict<br>

Parametros:

* sentences: placeholder del tamaño mas grande de una oracion en el dataset
* targets: placeholder del tamaño mas grande de un objetivo (entidad)
* aspects: placeholder del tamaño mas grande de un aspecto
* batch_size: cantidad de ejemplos de prueba utilizados en una iteracion

In [0]:
    def predict(self, sentences, targets, aspects, batch_size=None):
        assert len(sentences) == len(targets)
        assert len(sentences) == len(aspects)
        
        batches = self._get_mini_batch_start_end(len(sentences), batch_size)
        predictions, predictions_prob = [], []
        
        for start, end in batches:
            feed_dict = {
                self._sentences: sentences[start:end], 
                self._targets: targets[start:end],
                self._aspects: aspects[start:end],
                self._entnet_input_keep_prob: 1.0,
                self._entnet_output_keep_prob: 1.0,
                self._entnet_state_keep_prob: 1.0,
                self._final_layer_keep_prob: 1.0,
            }
            
            prediction, prediction_prob = self._sess.run(
                [self.predict_op, self.predict_proba_op],
                feed_dict=feed_dict
            )
            
            predictions.extend(prediction)
            predictions_prob.extend(prediction_prob)
            
        return predictions, np.array(predictions_prob)

# 9. Sentence length<br>

Parametros:

* sentences: oracion transformada en vectores de embedding

In [0]:
    def _sentence_length(self, sentences):
        '''
        sentences: (None, sentence_len, embedding_size)
        '''
        used = tf.sign(tf.reduce_max(tf.abs(sentences), reduction_indices=2))
        length = tf.reduce_sum(used, reduction_indices=1)
        length = tf.cast(length, tf.int32)
        return length

# ==================================================

# 10. Dynamic Memory Cell (Constructor)<br>

Parametros:

* (atributo) num_blocks: Cantidad de cadenas
* (atributo) num_units_per_block: Tamaño del vector de embeddings
* (atributo) keys: Embeddings de los keys de cada cadena
* (atributo) initializer: Forma de inicializar los pesos de la red neuronal
* (atributo) recurrent_initializer: Forma de inicializar los pesos de la red neuronal
* (atributo) activation: Funcion de activacion (phi) ReLU



In [0]:
class DynamicMemoryCell(tf.contrib.rnn.RNNCell):
    """
    Implementation of a dynamic memory cell as a gated recurrent network.
    The cell's hidden state is divided into blocks and each block's weights are tied.
    """

    def __init__(self,
                 num_blocks,
                 num_units_per_block,
                 keys,
                 initializer=None,
                 recurrent_initializer=None,
                 activation=tf.nn.relu,):
        
        self._num_blocks = num_blocks # M
        self._num_units_per_block = num_units_per_block # d
        self._keys = keys
        self._activation = activation # \phi
        self._initializer = initializer
        self._recurrent_initializer = recurrent_initializer

# 11. State size

Retorna el tamaño total de la celda

In [0]:
    @property
    def state_size(self):
        return self._num_blocks * self._num_units_per_block * 2

# 12. Output size

Retorna el tamaño total de la salida de la celda

In [0]:
    @property
    def output_size(self):
        return self._num_blocks * self._num_units_per_block

# 13. Zero state<br>

Parametros:

* batch_size: cantidad de ejemplos de entrenamiento utilizados en una iteracion

```
keys = [[1., 1., 1., 1.], [2., 2., 2., 2.]]
zero_state = [1., 1., 1., 1., 2., 2., 2., 2.]
batch_size = 5

zero_state_batch =[[1., 1., 1., 1., 2., 2., 2., 2.],
                   [1., 1., 1., 1., 2., 2., 2., 2.],
                   [1., 1., 1., 1., 2., 2., 2., 2.],
                   [1., 1., 1., 1., 2., 2., 2., 2.],
                   [1., 1., 1., 1., 2., 2., 2., 2.]]

num_blocks = 2
num_units_per_block = 4

return = [[1., 1., 1., 1., 2., 2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
          [1., 1., 1., 1., 2., 2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
          [1., 1., 1., 1., 2., 2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
          [1., 1., 1., 1., 2., 2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
          [1., 1., 1., 1., 2., 2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0.]]
```



In [0]:
    def zero_state(self, batch_size, dtype):
        "Initialize the memory to the key values."
        zero_state = tf.concat([tf.expand_dims(key, axis=0) for key in self._keys], axis=1)
        zero_state_batch = tf.tile(zero_state, [batch_size, 1])
        
        return tf.concat(
            values=[
                zero_state_batch,
                tf.zeros(
                    shape=[batch_size, self._num_blocks * self._num_units_per_block],
                    dtype=tf.float32,
                ),
            ],
            axis=1
        )

# 14. Get gate<br>

Parametros:

* state_j: memoria anterior a la actual
* key_j: embedding del objetivo (entidad) de la cadena de memorias
* inputs: palabras de la oracion de entrada
* v: vector entrenable
* prev_a: activacion previa

<img src="https://i.ibb.co/nQgMrKq/liu2018-eq2.png" alt="liu2018-eq2" border="0">

# $a  = w_{i} * h^{j}_{i-1} $
# $b  = w_{i} * k^{j} $
# $c  = v * d^{j}_{i} $

```
batch_size = 5
embedding_size = 4

a = inputs(4x1) * state_j(1x4) = (4x4)
b = inputs(4x1) * key_j(1x4) = (4x4)
c = prev_a(4x1) * v(1x4) = (4x4)

a + b + c = (4x4)

```

In [0]:
    def get_gate(self, state_j, key_j, inputs, v=None, prev_a=None):
        a = tf.reduce_sum(inputs * state_j, axis=1)
        b = tf.reduce_sum(inputs * key_j, axis=1)
        assert v is not None
        c = tf.reduce_sum(prev_a * v, axis=1)
        return tf.sigmoid(a + b + c)

# 15. Get candidate<br>

Parametros:

* state_j: memoria anterior a la actual
* key_j: embedding del objetivo (entidad) de la cadena de memorias
* inputs: palabras de la oracion de entrada
* U, V y W: matrices de parametros entrenables
* U_bias: bias de la matriz de parametros entrenable U

<img src="https://i.ibb.co/x7Fk6GC/liu2018-eq3y4.png" alt="liu2018-eq3y4" border="0">

# $ key = V k^{j} $
# $ state = U h^{j}_{i-1} $
# $ inputs = W w_{i} $

```
num_units_per_block = 4

key = V(4x4) * k(4x1) = (4x1)
state = U(4x4) * h(4x1) = (4x1) + bias(4x1) = (4x1)
inputs = W(4x4) * w(4x1) = (4x1)

state + inputs + key = (4x1)

return = [[15.],
          [15.],
          [15.],
          [15.]]

```

* U, V y W: matrices de tamaño (num_units_per_block x num_units_per_block)
* U_bias: bias de tamano (num_units_per_block x 1)

In [0]:
    def get_candidate(self, state_j, key_j, inputs, U, V, W, U_bias):
        key_V = tf.matmul(key_j, V)
        state_U = tf.matmul(state_j, U) + U_bias
        inputs_W = tf.matmul(inputs, W)
        
        return self._activation(state_U + inputs_W + key_V)

# 16. Dynamic Memory Cell (Funcion)

Parametros:

* inputs: palabras de la oracion de entrada
* state: memoria
* U, V y W: matrices de tamaño (num_units_per_block x num_units_per_block)
* U_bias: bias de tamano (num_units_per_block x 1)
* num_units_per_block: tamaño del vector de embeddings
* recurrent_initializer: forma de inicializar los pesos



In [0]:
    def __call__(self, inputs, state, scope=None):
        with tf.variable_scope(scope or type(self).__name__, initializer=self._initializer):
            U = tf.get_variable('U', [self._num_units_per_block, self._num_units_per_block],
                                initializer=self._recurrent_initializer)
            V = tf.get_variable('V', [self._num_units_per_block, self._num_units_per_block],
                                initializer=self._recurrent_initializer)
            W = tf.get_variable('W', [self._num_units_per_block, self._num_units_per_block],
                                initializer=self._recurrent_initializer)

            U_bias = tf.get_variable('U_bias', [self._num_units_per_block])

```
batch_size = 5
embedding_size = 4
num_blocks = 2
num_units_per_block = embedding_size = 4

state = [[1., 1., 1., 1., 2., 2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 2., 2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 2., 2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 2., 2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 2., 2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0.]]
         
state = [[1., 1., 1., 1., 2., 2., 2., 2.],
         [1., 1., 1., 1., 2., 2., 2., 2.],
         [1., 1., 1., 1., 2., 2., 2., 2.],
         [1., 1., 1., 1., 2., 2., 2., 2.],
         [1., 1., 1., 1., 2., 2., 2., 2.]]

state_a = [[0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.]]

```

In [0]:
            state, state_a = tf.split(
                value=state,
                num_or_size_splits=[
                    self._num_blocks * self._num_units_per_block,
                    self._num_blocks * self._num_units_per_block
                ],
                axis=1,
            )

```
state_a = [array([[0., 0., 0., 0.],
                  [0., 0., 0., 0.],
                  [0., 0., 0., 0.],
                  [0., 0., 0., 0.],
                  [0., 0., 0., 0.]], dtype=float32), 
           array([[0., 0., 0., 0.],
                  [0., 0., 0., 0.],
                  [0., 0., 0., 0.],
                  [0., 0., 0., 0.],
                  [0., 0., 0., 0.]], dtype=float32)]

```



In [0]:
            state_a = tf.split(state_a, self._num_blocks, axis=1)
            assert len(state_a) == self._num_blocks



```
state = [array([[1., 1., 1., 1.],
                [1., 1., 1., 1.],
                [1., 1., 1., 1.],
                [1., 1., 1., 1.],
                [1., 1., 1., 1.]], dtype=float32), 
         array([[2., 2., 2., 2.],
                [2., 2., 2., 2.],
                [2., 2., 2., 2.],
                [2., 2., 2., 2.],
                [2., 2., 2., 2.]], dtype=float32)]
```



In [0]:
            # Split the hidden state into blocks (each U, V, W are shared across blocks).
            state = tf.split(state, self._num_blocks, axis=1)
            assert len(state) == self._num_blocks

```
j = 0
state_j = [[1. 1. 1. 1.]
           [1. 1. 1. 1.]
           [1. 1. 1. 1.]
           [1. 1. 1. 1.]
           [1. 1. 1. 1.]]

j = 1
state_j = [[2. 2. 2. 2.]
           [2. 2. 2. 2.]
           [2. 2. 2. 2.]
           [2. 2. 2. 2.]
           [2. 2. 2. 2.]]
```

In [0]:
            next_states = []
            next_a_states = []
            for j, state_j in enumerate(state): # Hidden State (j)

```
keys = [[1., 1., 1., 1.], 
        [2., 2., 2., 2.]]

j = 0
key_j = [[1., 1., 1., 1.]]

j = 1
key_j = [[2., 2., 2., 2.]]

```

In [0]:
                key_j = tf.expand_dims(self._keys[j], axis=0)

* get_candidate: Seccion 15

```
candidate_j = [[15.],
               [15.],
               [15.],
               [15.]]
```

In [0]:
                candidate_j = self.get_candidate(state_j, key_j, inputs, U, V, W, U_bias)

                reuse = False
                if j != 0:
                    reuse = True

```
num_units_per_block = embedding_size = 4

w_ru = (4*2 x 4*2) = (8 x 8)
b_ru = (4*2 x 1)   = (8 x 1)
w_c  = (4*2 x 4)   = (8 x 4)
b_c  = (4 x 1)     = (4 x 1)
```

In [0]:
                with tf.variable_scope("entnet_gru", reuse=reuse) as gru_scope:
                    w_ru = tf.get_variable(
                        "w_ru", 
                        [self._num_units_per_block * 2, self._num_units_per_block * 2]
                    )
      
                    b_ru = tf.get_variable(
                        "b_ru", [self._num_units_per_block * 2],
                        initializer=init_ops.constant_initializer(1.0))
        
                    w_c = tf.get_variable("w_c",
                        [self._num_units_per_block * 2, self._num_units_per_block]
                    )
          
                    b_c = tf.get_variable(
                        "b_c", [self._num_units_per_block],
                        initializer=init_ops.constant_initializer(0.0))

* gru_block_cell: Celda GRU de tensorflow

In [0]:
                    _gru_block_cell = gen_gru_ops.gru_block_cell  # pylint: disable=invalid-name
  
                    _, _, _, new_a = _gru_block_cell(
                        x=candidate_j, h_prev=state_a[j], 
                        w_ru=w_ru, w_c=w_c, b_ru=b_ru, b_c=b_c)

```
num_units_per_block = embedding_size = 4
v_a = (4 x 1)
```

In [0]:
                    v_a = tf.get_variable(
                        "v_a", [self._num_units_per_block],
                        initializer=self._initializer,
                    )

* get_gate: Seccion 14

In [0]:
                next_a_states.append(new_a)

                gate_j = self.get_gate(state_j, key_j, inputs, v_a, new_a)

<img src="https://i.ibb.co/x8P2Lsr/liu2018-eq4.png" alt="liu2018-eq4" border="0">

In [0]:
                # Equation 4: h_j <- h_j + g_j * h_j^~
                # Perform an update of the hidden state (memory).
                state_j_next = state_j + tf.expand_dims(gate_j, -1) * candidate_j

In [0]:
                # Equation 5: h_j <- h_j / \norm{h_j}
                # Forget previous memories by normalization.
                state_j_next_norm = tf.norm(
                    tensor=state_j_next,
                    ord='euclidean',
                    axis=-1,
                    keep_dims=True)
                
                state_j_next_norm = tf.where(
                    tf.greater(state_j_next_norm, 0.0),
                    state_j_next_norm,
                    tf.ones_like(state_j_next_norm))
                
                state_j_next = state_j_next / state_j_next_norm

                next_states.append(state_j_next)
            
            state_next = tf.concat(next_states, axis=1)
            state_a_next = tf.concat(next_a_states, axis=1)
            
            return state_next, tf.concat(values=[state_next, state_a_next], axis=1)

# ==================================================

# 17. Zero nil slot<br>

Parametros:

* t: 

In [0]:
def zero_nil_slot(t, name=None):
    """
    Overwrites the nil_slot (first row) of the input Tensor with zeros.

    The nil_slot is a dummy slot and should not be trained and influence
    the training algorithm.
    """
    with name_scope(values=[t], name=name, default_name="zero_nil_slot") as name:
        t = tf.convert_to_tensor(t, name="t")
        s = tf.shape(t)[1]
        z = tf.zeros(tf.stack([1, s]))
        
        return tf.concat(
            axis=0, values=[z, tf.slice(t, [1, 0], [-1, -1])], name=name
        )

# 18. PReLU<br>

<a href="https://imgbb.com/"><img src="https://i.ibb.co/cbc8zdv/prelu.png" alt="prelu" border="0"></a>

$a_{i}$ es un coeficiente que controla la pendiente de la parte negativa. Si $a_{i}=0$ es la funcion ReLU.

Parametros:

* features: vector de caracteristicas
* alpha: coeficiente que controla la pendiente de la parte negativa
* scope: ambito

Funciones:

* tf.nn.relu: funcion de activacion ReLU

<img src="http://www.diegocalvo.es/wp-content/uploads/2018/12/funci%C3%B3n-ReLU-300x63.png" alt="relu" border="0">

In [0]:
def prelu(features, alpha, scope=None):
    """
    Implementation of [Parametric ReLU](https://arxiv.org/abs/1502.01852) borrowed from Keras.
    """
    with tf.variable_scope(scope, 'PReLU'):
        pos = tf.nn.relu(features)
        neg = alpha * (features - tf.abs(features)) * 0.5 
        # (features - tf.abs(features)) * 0.5 = features (si features es negativo)
        # (features - tf.abs(features)) * 0.5 = 0.0      (si features es positivo)
        return pos + neg